In [ ]:
import numpy as np
import scipy as scp
import pandas as pd

In [ ]:
# # Pre-processing Section

# path="./ml-latest/"

# # to do preprocessing uncomment the following paragraph and comment the immediate next like to it

# data=pd.read_csv(path+"ratings.csv")
# movie_ids=np.unique(data["movieId"].values)
# max_movie_id=np.max(movie_ids)
# user_ids=np.unique(data["userId"].values)
# max_user_id=np.max(user_ids)
# mat=np.zeros((max_movie_id+1,max_user_id+1))
# for _,i in data.iterrows():
#     mat[int(i[1])][int(i[0])]=i[2]






A_big=np.array(np.random.binomial(n=1,p=0.4,size=(10,10)),dtype=np.float64)
A_small=np.array(np.random.binomial(n=1,p=0.4,size=(5,5)),dtype=np.float64)


# Returns A_big,A_small matrix


In [ ]:

U,S,V_T=scp.sparse.linalg.svds(A_big)



loss_list_big_SVD=[]
for i in range(1,len(A_big)+1):
    A_hat=U[:,:i]@np.diag(S[:i])@V_T[:i,:]
    loss_list_big_SVD.append(np.linalg.norm(A_big-A_hat,ord='fro'))

# U,S,V_T=scp.sparse.linalg.svds(A_small)
U,S,V_T=np.linalg.svd(A_small)
loss_list_small_SVD=[]
for i in range(1,len(A_small)+1):
    A_hat=U[:,:i]@np.diag(S[:i])@V_T[:i,:]
    loss_list_small_SVD.append(np.linalg.norm(A_small-A_hat,ord='fro'))

loss_list_big_CUR=[]
for i in range(1,len(A_big)+1):
    C,U,R=CUR(A_big,i)
    A_hat=C@U@R
    loss_list_big_CUR.append(np.linalg.norm(A_big-A_hat,ord='fro'))

loss_list_small_CUR=[]
for i in range(1,len(A_small)+1):
    C,U,R=CUR(A_small,i)
    A_hat=C@U@R
    loss_list_small_CUR.append(np.linalg.norm(A_small-A_hat,ord='fro'))




In [ ]:
print(loss_list_big_SVD)
print(loss_list_big_CUR)
